In [1]:
from models import *
from collections import OrderedDict
import torch
import os
import numpy as np
import torch.nn as nn
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0")
target = VGG('VGG16')
pthfile = r"checkpoints/VGG16_ckpt.pth"
d = torch.load(pthfile)['net']
d = OrderedDict([(k[7:],v) for (k,v) in d.items()])
target.load_state_dict(d)
target = target.to(device)
dae = torch.load("checkpoints/model.pkl").to(device)
dae.target = nn.ReLU()

In [2]:
from utils import Dataset
attackPath = ["../adv_data/cifar10/fgsm/0.015x_adv.npy",
              "../adv_data/cifar10/fgsm/0.03x_adv.npy",
              "../adv_data/cifar10/fgsm/0.06x_adv.npy",
              "../adv_data/cifar10/cw/0.015x_adv.npy",
              "../adv_data/cifar10/cw/0.03x_adv.npy",
              "../adv_data/cifar10/cw/0.06x_adv.npy",
              "../adv_data/cifar10/bim/0.015x_adv.npy",
              "../adv_data/cifar10/bim/0.03x_adv.npy",
              "../adv_data/cifar10/bim/0.06x_adv.npy"
             ]
cleanPath = "data/cifar10_data.npy"
labelPath = "data/cifar10_label.npy"
dataset = Dataset(cleanPath = cleanPath, attackPath = attackPath, labelPath = labelPath,net = target)

In [73]:
# adv_data = dataset.attackTrain[8]
# batch = 50
# h = []
# for i in range(len(adv_data)//batch):
#     data = adv_data[i*batch:(i+1)*batch]
#     data = torch.from_numpy(data).to(device)
#     data = dae(data)
#     data = target.features(data)
#     data = data.detach().cpu().numpy().squeeze()
#     h.append(data)
# h = np.array(h)
# h = np.reshape(h,(-1,512))
# np.save("data/hidden_cifar10_vgg16_bim16.npy",h)

In [74]:
attacks = ["data/hidden_cifar10_vgg16_fgsm4.npy",
              "data/hidden_cifar10_vgg16_fgsm8.npy",
              "data/hidden_cifar10_vgg16_fgsm16.npy",
              "data/hidden_cifar10_vgg16_cw4.npy",
              "data/hidden_cifar10_vgg16_cw8.npy",
              "data/hidden_cifar10_vgg16_cw16.npy",
              "data/hidden_cifar10_vgg16_bim4.npy",
              "data/hidden_cifar10_vgg16_bim8.npy",
              "data/hidden_cifar10_vgg16_bim16.npy",
             ]
hidden_ = np.load("data/cifar10_vgg16_hidden.npy")
attack_data = [np.load(i) for i in attacks]

In [130]:
def hidden_dataset():
    for _ in range(1000):
        data = []
        label = []
        for i in range(50):
            s = random.randint(0,49999)
            a = random.randint(0,8)
            
            x1 = attack_data[a][s]
            x2 = hidden_[s]
            lamda = random.uniform(0,1)
            x = lamda*x1 + (1-lamda)*x2
            
            data.append(x1)
            data.append(x2)
            data.append(x)
            label.append(dataset.labelTrain[s])
            label.append(dataset.labelTrain[s])
            label.append(dataset.labelTrain[s])
            
        yield np.array(data),np.array(label)

In [ ]:
from models import HiddenDenoiser
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
# hidden_denoiser = HiddenDenoiser().to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, hidden_denoiser.parameters()))
for epoch in range(100):
    running_loss = 0.0
    dataGen = hidden_dataset()
    for i, data in enumerate(dataGen, 1):
        # get the inputs
        inputs, labels = data
        inputs, labels = torch.from_numpy(inputs).to(device),torch.from_numpy(labels).to(device)
        inputs, labels = Variable(inputs,requires_grad=True), Variable(labels)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward
        outputs = hidden_denoiser(inputs)
        outputs = target.classifier(outputs)
        # loss
        loss = nn.CrossEntropyLoss()(outputs, labels)
        # backward
        loss.backward()
        # update weights
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 200 == 0:  # print every 2000 mini-batches
            print('epoch: %d  loss: %.3f' %
                  (epoch + 1 , running_loss / 50))
            running_loss = 0.0
    print("evaluating")
    correct = 0
    total = 0
    testGen = dataset.testGen(8,clean = 1)
    for data in testGen:
        inputs, labels = data
        inputs,labels = torch.from_numpy(inputs),torch.from_numpy(labels)
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = dae(inputs)
        outputs = target.features(outputs).squeeze()
        outputs = hidden_denoiser(outputs)
        outputs = target.classifier(outputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy {}'.format(100. * correct / total))

epoch: 1  loss: 1.133
epoch: 1  loss: 1.136
epoch: 1  loss: 1.123
epoch: 1  loss: 1.104
epoch: 1  loss: 1.136
evaluating
Accuracy 90.5999984741211
epoch: 2  loss: 1.090
epoch: 2  loss: 1.148
epoch: 2  loss: 1.126
epoch: 2  loss: 1.126
epoch: 2  loss: 1.117
evaluating
Accuracy 90.8699951171875
epoch: 3  loss: 1.110
epoch: 3  loss: 1.091
epoch: 3  loss: 1.099
epoch: 3  loss: 1.111
epoch: 3  loss: 1.110
evaluating
Accuracy 90.75999450683594
epoch: 4  loss: 1.136
epoch: 4  loss: 1.141
epoch: 4  loss: 1.087
epoch: 4  loss: 1.092
epoch: 4  loss: 1.083
evaluating
Accuracy 90.72000122070312
epoch: 5  loss: 1.081
epoch: 5  loss: 1.112
epoch: 5  loss: 1.140
epoch: 5  loss: 1.062
epoch: 5  loss: 1.132
evaluating
Accuracy 90.81999969482422
epoch: 6  loss: 1.108
epoch: 6  loss: 1.082
epoch: 6  loss: 1.137
epoch: 6  loss: 1.109
epoch: 6  loss: 1.097
evaluating
Accuracy 90.69999694824219
epoch: 7  loss: 1.099
epoch: 7  loss: 1.110
epoch: 7  loss: 1.060
epoch: 7  loss: 1.096
epoch: 7  loss: 1.097
eval

In [133]:
correct = 0
total = 0
testGen = dataset.testGen(8,clean = 1)
for data in testGen:
    inputs, labels = data
    inputs,labels = torch.from_numpy(inputs),torch.from_numpy(labels)
    inputs, labels = inputs.to(device), labels.to(device)
    
    outputs = dae(inputs)
    outputs = target.features(outputs).squeeze()
#     outputs = hidden_denoiser(outputs)
    outputs = target.classifier(outputs)
    
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print('Accuracy {}'.format(100. * correct / total))

Accuracy 90.91999816894531
